In [1]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt

import time
from tqdm import tqdm

np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'E:\\User\\Stevens\\Spring 2024\\PTO - Fairness\\myGit\\myUtils')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
def genData(num_data, num_features, num_items, seed=42, Q=100, dim=1, deg=1, noise_width=0.5, epsilon=0.1):
    rnd = np.random.RandomState(seed)
    n = num_data
    p = num_features
    m = num_items
    
    # Split the population into group A (1/4) and group B (3/4)
    group_A_size = n // 4
    group_B_size = n - group_A_size
    
    # Generate x with a bias for group A
    x = np.zeros((n, m, p))
    x[:group_A_size] = rnd.normal(0.5, 1, (group_A_size, m, p))  # Slightly higher mean for group A
    x[group_A_size:] = rnd.normal(0, 1, (group_B_size, m, p))   # Standard distribution for group B
    
    B = rnd.binomial(1, 0.5, (m, p))

    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            values = (np.dot(B[j], x[i, j].reshape(p, 1)).T / np.sqrt(p) + 3) ** deg + 1
            values *= 5
            values /= 3.5 ** deg
            epislon = rnd.uniform(1 - noise_width, 1 + noise_width, 1)
            values *= epislon
            
            # Introduce bias for c: Group A has slightly higher values, Group B slightly lower
            if i < group_A_size:
                values *= 1.1  # Increase c for Group A
            else:
                values *= 0.9  # Decrease c for Group B
            
            values = np.ceil(values)
            c[i, j] = values

    c = c.astype(np.float64)
    r = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            # Generate r using normal distribution, then clip to [0, 1] range
            r[i, j] = np.clip(rnd.normal(0.5, 0.2), 0, 1)
            
            # Introduce bias for r: Group A has slightly lower values, Group B slightly higher
            if i < group_A_size:
                r[i, j] -= 0.1  # Decrease r for Group A
            else:
                r[i, j] += 0.1  # Increase r for Group B
            
            # Clip again to ensure values remain in [0, 1] range
            r[i, j] = np.clip(r[i, j], 0, 1)

    return x, r, c, Q

In [3]:
def visLearningCurve(loss_log, loss_log_regret, mse_loss_log):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 4))

    # Plot original loss log
    ax1.plot(loss_log, color="c", lw=2)
    ax1.tick_params(axis="both", which="major", labelsize=12)
    ax1.set_xlabel("Iters", fontsize=16)
    ax1.set_ylabel("Loss", fontsize=16)
    ax1.set_title("Learning Curve (Training Loss)", fontsize=16)

    # Plot regret log
    ax2.plot(loss_log_regret, color="royalblue", ls="--", alpha=0.7, lw=2)
    ax2.tick_params(axis="both", which="major", labelsize=12)
    ax2.set_xlabel("Epochs", fontsize=16)
    ax2.set_ylabel("Regret", fontsize=16)
    ax2.set_title("Learning Curve (Test Regret)", fontsize=16)

    # Plot new MSE loss log
    ax3.plot(mse_loss_log, color="orange", lw=2)
    ax3.tick_params(axis="both", which="major", labelsize=12)
    ax3.set_xlabel("Iters", fontsize=16)
    ax3.set_ylabel("MSE Loss", fontsize=16)
    ax3.set_title("Learning Curve (MSE Loss)", fontsize=16)

    plt.tight_layout()
    plt.show()

def objValue(d, r, alpha=0.5):
    """
    A function to calculate objective value
    """
    if alpha == 1:
        return np.sum(np.log(np.multiply(r, d)))
    else:
        return np.sum(np.power(np.multiply(r, d), 1 - alpha)) / (1 - alpha)

In [7]:
from scipy.linalg import solve

# Helper function to check matrix invertibility
def is_invertible(matrix):
    return np.linalg.cond(matrix) < 1 / np.finfo(matrix.dtype).eps

def solve_rd_matrix_system(r, c, d_i, mu, lambda_i, Q, alpha=0.5):
    n = len(r)
    
    # Step 1: Construct Hessian of the objective function (H_dd)
    A_elements = [alpha * r[i]**2 * (r[i] * d_i[i])**(-alpha - 1) for i in range(n)]
    H_dd = np.diag(A_elements)
    
    # Step 2: Construct Jacobians of inequality constraints (g(d;r)) and equality constraints (h(d;r))
    B = -np.eye(n)  # Jacobian of g(d;r) = d >= 0 is -I
    C = c.reshape(-1, 1)  # Jacobian of h(d;r) = sum(c * d) - Q is the vector of c_i's
    
    # Step 3: Construct D and M matrices
    D = np.diag(d_i)  # Diagonal matrix with d_i's
    M = mu * c.reshape(1, -1)  # Row vector of mu * c
    
    # Step 4: Complementary slackness matrix with lambda_i
    if lambda_i.ndim > 2:
        raise ValueError(f"lambda_i has too many dimensions: {lambda_i.ndim}")
    lambda_i = lambda_i.flatten()  # Ensure it's 1-dimensional
    Lambda = np.diag(lambda_i)  # Diagonal matrix from lambda_i
    
    # Step 5: Construct the full block matrix system
    LHS = np.block([
        [H_dd, B, C],  # First row block
        [Lambda, D, np.zeros((n, 1))],  # Second row block
        [M, np.zeros((1, n)), np.array([[np.sum(c * d_i) - Q]])]  # Third row block
    ])
    
    # Step 6: Construct the RHS vector
    v = np.array([-alpha * r[i] * (r[i] * d_i[i])**(-alpha - 1) for i in range(n)])
    RHS = np.hstack([v, np.zeros(n), np.zeros(1)])
    
    # Step 7: Solve the matrix system
    if is_invertible(LHS):
        solution = solve(LHS, RHS)
        d_r_derivatives = solution[:n]
        return d_r_derivatives
    else:
        return {"error": "LHS matrix is not invertible. Check problem formulation or input data."}


In [11]:
class optModelMatrix:
    """
    This is a class for optimization models.
    """

    def __init__(self, r, c, Q, alpha=0.5):
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data = num_data
        self.num_items = num_items

        
    def __repr__(self):
        return 'optModel ' + self.__class__.__name__

    def setObj(self, r, c):

        if self.alpha == 1:
            self.objective = cp.sum(cp.log(cp.multiply(r, self.d)))
        else:
            self.objective = cp.sum(cp.power(cp.multiply(r, self.d), 1 - self.alpha)) / (1 - self.alpha)
        
        self.constraints = [
            self.d >= 0,
            cp.sum(cp.multiply(c, self.d)) <= self.Q
        ]
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)


    def solve(self, closed=False):
        """
        A method to solve the optimization problem for one set of parameters.

        Args:
            r (np.ndarray): The r parameter for the optimization
            c (np.ndarray): The c parameter for the optimization
            closed (bool): solving the problem in closed form

        Returns:
            tuple: optimal solution and optimal value
        """
        if closed:
            return self.solveC()

        self.d = cp.Variable(self.num_items)
        self.setObj(self.r, self.c)
        self.problem.solve(abstol=1e-9, reltol=1e-9, feastol=1e-9)
        opt_sol = self.d.value
        opt_val = self.problem.value


        # Dual values for the constraints
        lambdas = self.constraints[0].dual_value
        mus = self.constraints[1].dual_value
    
        return opt_sol, opt_val, lambdas, mus
    
class optDatasetMatrix(Dataset):
    """
    This class is Torch Dataset class for optimization problems.
    """

    def __init__(self, features, costs, r, Q, alpha=0.5, closed=False):
        """
        A method to create a optDataset from optModel

        Args:
            model (optModel): optimization model
            features (np.ndarray): features
            c (np.ndarray): c of objective function
            r (np.ndarray): r of objective function
            Q (float): budget
            alpha (float): alpha of objective function
            closed (bool): solving the problem in closed form

        """
        self.feats = features
        self.costs = costs
        self.r = r
        self.Q = Q
        self.alpha = alpha
        self.closed = closed
        # Now store the dual values
        self.sols, self.objs, self.lambdas, self.mus = self._getSols()

    def _getSols(self):
        """
        A method to get the solutions and dual values of the optimization problem
        """
        opt_sols = []
        opt_objs = []
        dual_lambdas = []
        dual_mus = []
        
        for i in tqdm(range(len(self.costs))):
            sol, obj, lambdas, mus = self._solve(self.r[i], self.costs[i])
            opt_sols.append(sol)
            opt_objs.append([obj])
            dual_lambdas.append(lambdas)
            dual_mus.append(mus)
            
        return np.array(opt_sols), np.array(opt_objs), np.array(dual_lambdas), np.array(dual_mus)


    def  _solve(self, r, c):
        """
        A method to solve the optimization problem to get oan optimal solution with given r and c

        Args:
            r (np.ndarray): r of objective function
            c (np.ndarray): c of objective function

        Returns:
            tuple: optimal solution (np.ndarray), objective value (float)
        """
        self.model = optModelMatrix(r, c, self.Q, self.alpha)
        if self.closed:
            return self.model.solveC()
        else:
            return self.model.solve()

    def __len__(self):
        """
        A method to get data size

        Returns:
            int: the number of optimization problems
        """
        return len(self.costs)
    
    def __getitem__(self, index):
        return (
            torch.FloatTensor(self.feats[index]),  # x 
            torch.FloatTensor(self.costs[index]),  # c
            torch.FloatTensor(self.r[index]),      # r 
            torch.FloatTensor(self.sols[index]),   # optimal solution
            torch.FloatTensor(self.objs[index]),   # objective value
            torch.FloatTensor(self.lambdas[index]),  # dual value (lambdas)
            torch.FloatTensor([self.mus[index]])     # dual value (mus)
        )

In [12]:
num_data = 1000
num_features = 20
num_items = 5
x, r, c, Q = genData(num_data, num_features, num_items)
# create optmodel instance
optmodel = optModelMatrix(r, c, Q, alpha=0.5)

In [13]:
optmodel = optModelMatrix(r, c, Q, alpha=0.5)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test, r_train, r_test = train_test_split(x, c, r, test_size=0.2, random_state=42)

# Create datasets and dataloaders
dataset_train = optDatasetMatrix(x_train, c_train, r_train, Q, alpha=0.5, closed=False)
dataset_test = optDatasetMatrix(x_test, c_test, r_test, Q, alpha=0.5, closed=False)
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

100%|██████████| 200/200 [00:00<00:00, 298.93it/s]
